In [200]:
import pandas as pd
import numpy as np
import requests 
from datetime import datetime, date


## 1. data load

In [201]:
# 1. data load
data = pd.read_excel("/Users/yj.noh/Desktop/seoul/train_data_new.xlsx")
data.rename(columns = {'라이더수':'rider_cnt', '주문수':'order_cnt'}, inplace = True)

print(data.shape) 

(83975, 7)


In [202]:
# 2. seoul, 9~23시만
# Filter data
data['reg_date'] = pd.to_datetime(data['reg_date'])
data = data[data['pick_rgn1_nm'] == '서울특별시']
data = data[data['reg_date'] < data['reg_date'].max()]
data = data[data['hour_reg'].isin(range(9, 24))]

print(data.shape) 

(72375, 7)


## 2. 파생변수 생성

### 2-1. datetime 

In [203]:
# datetime 컬럼 만들기 
data['reg_date'] = pd.to_datetime(data['reg_date'])
data['datetime'] = data['reg_date'] + pd.to_timedelta(data['hour_reg'], unit='h')
data['hour_reg2'] = data['datetime'].dt.hour
data['reg_date2'] = data['datetime'].dt.date

print(data['datetime'].min(), data['datetime'].max())

2022-12-01 09:00:00 2023-06-11 23:00:00


In [204]:
data['weekday'] = data['reg_date'].dt.weekday

weekday_dict = {0:'월요일', 1:'화요일', 2:'수요일', 3:'목요일', 4:'금요일', 5:'토요일', 6:'일요일'}
data['day_of_reg'] = data['weekday'].map(weekday_dict)

print(data)

        reg_date  hour_reg holiday_yn pick_rgn1_nm pick_rgn2_nm  rider_cnt   
0     2022-12-01         9          N        서울특별시          금천구         23  \
1     2022-12-01         9          N        서울특별시          서초구         74   
2     2022-12-01         9          N        서울특별시          송파구         89   
3     2022-12-01         9          N        서울특별시          용산구         34   
4     2022-12-01         9          N        서울특별시          강남구        190   
...          ...       ...        ...          ...          ...        ...   
83956 2023-06-11        23          Y        서울특별시          강동구         93   
83957 2023-06-11        23          Y        서울특별시         동대문구        119   
83958 2023-06-11        23          Y        서울특별시         서대문구        104   
83959 2023-06-11        23          Y        서울특별시          성북구        122   
83960 2023-06-11        23          Y        서울특별시          도봉구         57   

       order_cnt            datetime  hour_reg2   reg_date2  we

In [205]:
data.drop(['reg_date', 'hour_reg', 'pick_rgn1_nm', 'holiday_yn','weekday'], axis=1, inplace=True)      
data.rename(columns={"hour_reg2": "hour_reg", "reg_date2": "reg_date"}, inplace=True)


print(data.dtypes)
print(data.head(3))

pick_rgn2_nm            object
rider_cnt                int64
order_cnt                int64
datetime        datetime64[ns]
hour_reg                 int32
reg_date                object
day_of_reg              object
dtype: object
  pick_rgn2_nm  rider_cnt  order_cnt            datetime  hour_reg   
0          금천구         23         34 2022-12-01 09:00:00         9  \
1          서초구         74        108 2022-12-01 09:00:00         9   
2          송파구         89        177 2022-12-01 09:00:00         9   

     reg_date day_of_reg  
0  2022-12-01        목요일  
1  2022-12-01        목요일  
2  2022-12-01        목요일  


### 2-2. 날씨 기상 여부 붙이기 

In [206]:
# weather
# Read csv
weather = pd.read_csv("/Users/yj.noh/Desktop/seoul/weather_new.csv", encoding='cp949')

weather.rename(columns={"기온(°C)": "temp_c", "강수량(mm)": "rain_c", "적설(cm)": "snow_c", "일시": "date"}, inplace=True)

weather['date_2'] = pd.to_datetime(weather['date']).dt.date
weather['hour'] = pd.to_datetime(weather['date']).dt.hour

#print(weather['hour'].value_counts())
print(weather.head())

    지점 지점명              date  temp_c  rain_c  snow_c      date_2  hour
0  108  서울  2022-12-01 00:00    -8.0     NaN     NaN  2022-12-01     0
1  108  서울  2022-12-01 01:00    -7.7     NaN     NaN  2022-12-01     1
2  108  서울  2022-12-01 02:00    -7.5     NaN     NaN  2022-12-01     2
3  108  서울  2022-12-01 03:00    -7.7     NaN     NaN  2022-12-01     3
4  108  서울  2022-12-01 04:00    -7.9     NaN     NaN  2022-12-01     4


In [207]:

# Filter hours
weather = weather[weather['hour'].isin([9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])]

# Join
combined_data = pd.merge(data, weather[["date_2","hour","temp_c","rain_c", "snow_c"]], left_on=["reg_date", "hour_reg"], right_on=["date_2", "hour"], how='left')

# Fill NA
combined_data['rain_c'].fillna(0, inplace=True)
combined_data['snow_c'].fillna(0, inplace=True)

combined_data.drop(['hour', 'date_2', 'temp_c'], axis=1, inplace=True)    
print(combined_data.isnull().sum())

pick_rgn2_nm    0
rider_cnt       0
order_cnt       0
datetime        0
hour_reg        0
reg_date        0
day_of_reg      0
rain_c          0
snow_c          0
dtype: int64


In [208]:
# 변수 생성 - is_rain
combined_data['is_rain'] = combined_data.apply(lambda row: 1 if row['rain_c'] > 0 or row['snow_c'] > 0 else 0, axis=1)
print(combined_data['is_rain'].value_counts()) 

print(combined_data['datetime'].min()) 
print(combined_data['datetime'].max()) 


is_rain
0    63275
1     9100
Name: count, dtype: int64
2022-12-01 09:00:00
2023-06-11 23:00:00


In [209]:
# is_rain2 
combined_data = combined_data.groupby('reg_date').apply(lambda x: x.assign(is_rain2 = 1 if x['is_rain'].sum() > 0 else 0)).reset_index(drop=True)


### 2-4. is_holiday

In [210]:
# 공휴일 유무 
holiday_list = pd.to_datetime(['2022-01-01', '2022-01-31', '2022-02-01', '2022-03-01', '2022-03-09', '2022-05-05', '2022-05-08', '2022-06-01', '2022-06-06', '2022-08-15', 
                '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2022-10-10', '2022-12-25', '2023-01-01', '2023-01-21', 
                '2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01', '2023-05-01', '2023-05-05', '2023-05-27', '2023-05-29', '2023-06-06', '2023-08-15', 
                '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25'])

combined_data['reg_date'] = pd.to_datetime(combined_data['reg_date'])
combined_data['is_holiday'] = combined_data.apply(lambda row: 1 if (row['reg_date'] in holiday_list) or (row['day_of_reg'] in ['토요일', '일요일']) else 0, axis=1)

print(combined_data['is_holiday'].value_counts()) 

is_holiday
0    48750
1    23625
Name: count, dtype: int64


## 3. outlier 

In [211]:
# 이상치(outlier) 여부 파악 
def calculate_quantiles(group):
    q1 = group['rider_cnt'].quantile(0.25)
    q3 = group['rider_cnt'].quantile(0.75)
    IQR1_5 = 1.5 * (q3 - q1)
    group['q1'] = q1
    group['q3'] = q3
    group['IQR1.5'] = IQR1_5
    group['outlier'] = np.where(((group['is_rain'] == 0) & ((group['rider_cnt'] < (q1 - IQR1_5)) | (group['rider_cnt'] > (q3 + IQR1_5))) | 
                               (group['is_holiday'] == 0) & ((group['rider_cnt'] < (q1 - IQR1_5)) | (group['rider_cnt'] > (q3 + IQR1_5))))  , 1, 0)
    return group



In [212]:
train_data = combined_data[combined_data['reg_date'] <= pd.to_datetime('2023-03-31')]
test_data = combined_data[combined_data['reg_date'] >= pd.to_datetime('2023-04-01')]

print(train_data['reg_date'].min()) #2022-12-01
print(train_data['reg_date'].max()) #2023-03-31

print(test_data['reg_date'].min()) #2023-04-01
print(test_data['reg_date'].max()) #2023-06-11

2022-12-01 00:00:00
2023-03-31 00:00:00
2023-04-01 00:00:00
2023-06-11 00:00:00


In [213]:
train_data = train_data.groupby(['pick_rgn2_nm', 'day_of_reg', 'hour_reg', 'is_rain']).apply(calculate_quantiles)

print(train_data['outlier'].value_counts()) 

outlier
0    43109
1     2266
Name: count, dtype: int64


In [214]:
# outlier -> median 값으로 대체 

train_data = train_data.reset_index(drop=True)

train_data['rider_cnt_2'] = train_data['rider_cnt'].copy()
train_data['rider_cnt_2'] = train_data.groupby(['pick_rgn2_nm', 'day_of_reg', 'hour_reg'])['rider_cnt'].transform(lambda x: x.median() if 'outlier' in train_data.columns and train_data.loc[x.index, 'outlier'].any() else x)


In [215]:
# train data, test data 합치기

train_data = train_data.drop(['rider_cnt', 'q1', 'q3', 'IQR1.5', 'outlier'], axis=1)

train_data = train_data.rename(columns={'rider_cnt_2': 'rider_cnt'})
train_data = train_data[['pick_rgn2_nm', 'rider_cnt', 'order_cnt', 'datetime', 'hour_reg', 'reg_date', 'day_of_reg', 'rain_c', 'snow_c', 'is_rain', 'is_rain2', 'is_holiday']]

combined_data = pd.concat([train_data, test_data], ignore_index=True)


In [217]:
print(combined_data.head(3))
print(combined_data.isnull().sum())

  pick_rgn2_nm  rider_cnt  order_cnt            datetime  hour_reg   reg_date   
0          강남구      213.0        455 2022-12-02 09:00:00         9 2022-12-02  \
1          강남구      197.0        448 2022-12-09 09:00:00         9 2022-12-09   
2          강남구      180.0        393 2022-12-30 09:00:00         9 2022-12-30   

  day_of_reg  rain_c  snow_c  is_rain  is_rain2  is_holiday  
0        금요일     0.0     0.0        0         0           0  
1        금요일     0.0     0.0        0         0           0  
2        금요일     0.0     0.0        0         0           0  
pick_rgn2_nm    0
rider_cnt       0
order_cnt       0
datetime        0
hour_reg        0
reg_date        0
day_of_reg      0
rain_c          0
snow_c          0
is_rain         0
is_rain2        0
is_holiday      0
dtype: int64


### 4. w_1, w_2 rider수 변수 만들기 

In [ ]:
combined_data = combined_data.sort_values(['reg_date', 'pick_rgn2_nm'])
combined_data['rider_cnt_w_1'] = combined_data.groupby(['pick_rgn2_nm', 'day_of_reg', 'hour_reg','is_rain'])['rider_cnt'].shift(1)
combined_data['rider_cnt_w_2'] = combined_data.groupby(['pick_rgn2_nm', 'day_of_reg', 'hour_reg','is_rain'])['rider_cnt'].shift(2)
combined_data['rider_cnt_w_4'] = combined_data.groupby(['pick_rgn2_nm', 'day_of_reg', 'hour_reg','is_rain'])['rider_cnt'].shift(3)

print(new_data.isna().sum())